In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
timeMatrix = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/timeMatrixInflated.csv', index_col=0)

In [74]:
zones = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/poco-allzones.csv', index_col=0)
zones.index = zones.index + ', Port Coquitlam, BC, Canada'

In [75]:
zones = zones[(zones['Garbage Zone'] == 1)]

In [76]:
row = pd.Series({'Garbage Zone':1.0},name='1737 Broadway St Port Coquitlam, BC, Canada')

In [77]:
zones = zones.append(row)

In [78]:
timeMatrixFiltered = timeMatrix.iloc[timeMatrix.index.isin(zones.index),timeMatrix.index.isin(zones.index)]

In [79]:
# elements = list(range(0,len(timeMatrixFiltered)))
elements = list(range(0,16))
elements.append(1952)

In [80]:
timeMatrixFiltered = timeMatrixFiltered.iloc[elements,elements]

In [81]:
np.where(timeMatrixFiltered.index == '1737 Broadway St Port Coquitlam, BC, Canada')[0][0]

16

In [82]:
timeMatrix.index[41]

'1735 LINCOLN AVE, Port Coquitlam, BC, Canada'

In [83]:
timeWindows = []
for i in range(len(timeMatrixFiltered)):
    house = timeMatrixFiltered.index[i].split(' ')
    street = ' '.join(house[1:-4])
    
 
    if street == 'LINCOLN AVE,':
        timeWindows.append((0,50000))
    else:
        timeWindows.append((0,50000))
        
# timeWindows = []
# for i in range(17):
#     house = timeMatrixFiltered.index[i].split(' ')
#     street = ' '.join(house[1:-4])
    
 
#     if street == 'LINCOLN AVE,':
#         timeWindows.append((0,30))
#     else:
#         timeWindows.append((0,50000))

In [84]:
len(timeWindows)

17

In [85]:
np.where(timeMatrixFiltered.index == '1737 Broadway St Port Coquitlam, BC, Canada')

(array([16], dtype=int64),)

In [88]:
"""Vehicles Routing Problem (VRP) with Time Windows."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# def create_data_model(timeMatrix, numVehicles, depotLocation, timeWindows):
#     """Stores the data for the problem."""
#     data = {'time_matrix': timeMatrix.to_numpy(), 
#             'num_vehicles': numVehicles, 
#             'depot': depotLocation, 
#             'time_windows':timeWindows}
#     return data

def solver_status(status):
    if status == 0:
        return 'ROUTING_NOT_SOLVED: Problem not solved yet'
    elif status == 1:
        return 'ROUTING_SUCCESS: Problem solved successfully'
    elif status == 2:
        return 'ROUTING_FAIL: No solution found to the problem'
    elif status == 3:
        return 'ROUTING_FAIL_TIMEOUT: Time limit reached before finding a solution'
    elif status == 4:
        return 'ROUTING_INVALID: Model, model parameters, or flags are not valid'
    else:
        return "Unknown Status"


def create_data_model(timeMatrix, numVehicles, depot, timeWindows):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = timeMatrix.to_numpy()
    data['time_windows'] = timeWindows
    data['num_vehicles'] = numVehicles
    data['depot'] = depot
    return data


def print_solution(data, manager, routing, solution):
    routes = {}
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        routes[vehicle_id] = []
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            routes[vehicle_id].append(manager.IndexToNode(index))
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        routes[vehicle_id].append(manager.IndexToNode(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))
    return routes


def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model(timeMatrixFiltered,3, 16, timeWindows)
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.SAVINGS)
    
    search_parameters.time_limit.seconds = 300

    search_parameters.solution_limit = 1

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    print("Solver status:", solver_status(routing.status()))

    # Print solution on console.
    if solution:
        return print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    routes = main()

Solver status: ROUTING_SUCCESS: Problem solved successfully
Route for vehicle 0:
16 Time(0,0) -> 15 Time(371,371) -> 5 Time(435,435) -> 7 Time(481,481) -> 0 Time(556,556) -> 10 Time(648,648) -> 13 Time(864,864) -> 9 Time(976,976) -> 6 Time(991,991) -> 1 Time(1107,1107) -> 4 Time(1156,1156) -> 11 Time(1227,1227) -> 2 Time(1309,1309) -> 14 Time(1833,1833) -> 3 Time(2025,2025) -> 12 Time(2181,2181) -> 8 Time(2324,2324) -> 16 Time(2885,2885)
Time of the route: 2885min

Route for vehicle 1:
16 Time(0,0) -> 16 Time(0,0)
Time of the route: 0min

Route for vehicle 2:
16 Time(0,0) -> 16 Time(0,0)
Time of the route: 0min

Total time of all routes: 2885min


In [89]:
routes

{0: [16, 15, 5, 7, 0, 10, 13, 9, 6, 1, 4, 11, 2, 14, 3, 12, 8, 16],
 1: [16, 16],
 2: [16, 16]}

In [90]:
def route_names(routes, locations):
    namedRoutes = {}
    for vehicle in routes.keys():
        namedRoutes[vehicle] = locations[routes[vehicle]]
    return namedRoutes
    
namedRoutes = route_names(routes, timeMatrixFiltered.index)

In [91]:
namedRoutes[1]

Index(['1737 Broadway St Port Coquitlam, BC, Canada', '1737 Broadway St Port Coquitlam, BC, Canada'], dtype='object')

In [92]:
locations = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/locations.csv', index_col=0)

In [93]:
tableauRoutes = pd.DataFrame(columns={'address','latitude','longitude','pairID','vehicleID'})
for vehicle in namedRoutes.keys():
    for vertex in range(len(namedRoutes[vehicle])-1):
        currentVertexInfo = locations[locations.index == namedRoutes[vehicle][vertex]]
        currentVertex = {'address':currentVertexInfo.index[0],
                       'latitude':currentVertexInfo['lat'].values[0],
                       'longitude':currentVertexInfo['long'].values[0],
                       'pairID':vertex,
                       'vehicleID':vehicle}

        nextVertexInfo = locations[locations.index == namedRoutes[vehicle][vertex+1]]

        nextVertex = {'address':nextVertexInfo.index[0],
                       'latitude':nextVertexInfo['lat'].values[0],
                       'longitude':nextVertexInfo['long'].values[0],
                       'pairID':vertex,
                       'vehicleID':vehicle}

        tableauRoutes = tableauRoutes.append([currentVertex, nextVertex],ignore_index=True)
        
    
        
        

        
        
    

In [95]:
tableauRoutes.to_csv('timeWindows.csv')